In [1]:
print("ok")

ok


In [3]:
%pwd

'e:\\Medibot'

In [2]:
import os
os.chdir("../")

In [ ]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [8]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [12]:
extracted_data=load_pdf_file(data='E:\Medibot\Data')

In [13]:
def text_split(extracted_data):
    text_spiltter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spiltter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunks=text_split(extracted_data)
print("Length of Text chunks",len(text_chunks))

Length of Text chunks 5860


In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [22]:
embeddings=download_hugging_face_embedding()

e:\Medibot\medibot\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SMTA\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP do

In [23]:
query_result=embeddings.embed_query('apple')
print('Length',len(query_result))

Length 384


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [31]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec

pc=pinecone(api_key=PINECONE_API_KEY)
index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-qrim1iv.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [32]:
import os 
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [33]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [34]:
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [35]:
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type='similarity',search_kwargs={"k":3})

In [ ]:
retrieved_docs= retriever.invoke("what s Acne?") 